<a href="https://colab.research.google.com/github/revirevy/Airbnb_SF_EDA/blob/master/10k_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install llama-index  -q
#<0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.5/864.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00


In [7]:
# download files
!mkdir data
!wget https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1 -O data/UBER.zip
!unzip data/UBER.zip -d data

--2023-10-04 12:46:48--  https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/948jr9cfs7fgj99/UBER.zip [following]
--2023-10-04 12:46:49--  https://www.dropbox.com/s/dl/948jr9cfs7fgj99/UBER.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd2138355137a48351baf05a31f.dl.dropboxusercontent.com/cd/0/get/CE-zU9hlmUsfYaiE4J5-lZ2d8fT3Hg3DK_k5X-lRyZfmHFCCOh0ENaDG2k_U9MS0KoshbB0XGDr1dEaYWSROSl3GAr2_Hddayg5cg9aYzFxAMxvvPeylsCUP8ZaIfM4HN6OEXJIiAx-wRn3QPzmP4Iog/file?dl=1# [following]
--2023-10-04 12:46:49--  https://ucd2138355137a48351baf05a31f.dl.dropboxusercontent.com/cd/0/get/CE-zU9hlmUsfYaiE4J5-lZ2d8fT3Hg3DK_k5X-lRyZfmHFCCOh0ENaDG2k_U9MS0KoshbB0XGDr1dEaYWSROSl3GAr2_Hddayg5cg9aYzFxAMx

In [8]:
# set text wrapping
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [10]:
import os
# os.environ['OPENAI_API_KEY'] = ""

In [11]:
import llama_index
dir(llama_index)

['BaseCallbackHandler',
 'BasePromptTemplate',
 'BeautifulSoupWebReader',
 'ChatPromptTemplate',
 'ChromaReader',
 'ComposableGraph',
 'DeepLakeReader',
 'DiscordReader',
 'Document',
 'DocumentSummaryIndex',
 'EmptyIndex',
 'FaissReader',
 'GPTDocumentSummaryIndex',
 'GPTEmptyIndex',
 'GPTIndexMemory',
 'GPTKeywordTableIndex',
 'GPTKnowledgeGraphIndex',
 'GPTListIndex',
 'GPTPandasIndex',
 'GPTRAKEKeywordTableIndex',
 'GPTSQLStructStoreIndex',
 'GPTSimpleKeywordTableIndex',
 'GPTTreeIndex',
 'GPTVectorStoreIndex',
 'GithubRepositoryReader',
 'GoogleDocsReader',
 'IndexStructType',
 'JSONReader',
 'KeywordExtractPrompt',
 'KeywordTableIndex',
 'KnowledgeGraphIndex',
 'LLMPredictor',
 'LangchainEmbedding',
 'ListIndex',
 'MboxReader',
 'MilvusReader',
 'MockEmbedding',
 'MockLLMPredictor',
 'NotionPageReader',
 'NullHandler',
 'ObsidianReader',
 'OpenAIEmbedding',
 'Optional',
 'PandasIndex',
 'Path',
 'PineconeReader',
 'Prompt',
 'PromptHelper',
 'PromptTemplate',
 'PsychicReader',
 '

In [12]:
from llama_index import GPTVectorStoreIndex as  GPTSimpleVectorIndex, download_loader
from pathlib import Path

### Ingest Unstructured Data Through the Unstructured.io Reader

Leverage the capabilities of Unstructured.io HTML parsing.
Downloaded through LlamaHub.

In [13]:
UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)

In [14]:
loader = UnstructuredReader()
doc_set = {}
all_docs = []
years = [2022, 2021, 2020, 2019]
for year in years:
    year_docs = loader.load_data(file=Path(f'./data/UBER/UBER_{year}.html'), split_documents=False)
    # insert year metadata into each year
    for d in year_docs:
        d.extra_info = {"year": year}
    doc_set[year] = year_docs
    all_docs.extend(year_docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Setup Service Context

In [24]:
%pip install llama-cpp-python -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.3 MB/s eta 0:00:00


In [25]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=512,  ) # chunk_size_limit=512)

******
Could not load OpenAI model. Using default LlamaCPP=llama2-13b-chat. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******
total size (MB): 7365.83


7025it [00:51, 136.06it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


******
Could not load OpenAIEmbedding. Using HuggingFaceBgeEmbeddings with model_name=BAAI/bge-small-en. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


### Setup a Vector Index for each SEC filing

We setup a separate vector index for each SEC filing from 2019-2022.

We also optionally initialize a "global" index by dumping all files into the vector store.

In [41]:
[x for x in cur_index.__dir__() if 'write' in x]

[]

In [42]:
import pickle

In [43]:
# initialize simple vector indices + global vector index
# NOTE: don't run this cell if the indices are already loaded!
index_set = {}
for year in years:
    cur_index = GPTSimpleVectorIndex.from_documents(doc_set[year], service_context=service_context)
    index_set[year] = cur_index
    # cur_index.save(f'index_{year}.json')
    with open(f'index_{year}.json', 'wb') as file:
      pickle.dump(cur_index, file)

TypeError: ignored

In [ ]:
# Load indices from disk
index_set = {}
for year in years:
    cur_index = GPTSimpleVectorIndex.load_from_disk(f'index_{year}.json', service_context=service_context)
    index_set[year] = cur_index

In [ ]:
# NOTE: this global index is a single vector store containing all documents
# Only relevant for the section below: "Can a single vector index answer questions across years?"
global_index = GPTSimpleVectorIndex.from_documents(all_docs, service_context=service_context)
global_index.save_to_disk(f'index_global.json')

In [ ]:
global_index = GPTSimpleVectorIndex.load_from_disk(f'index_global.json', service_context=service_context)

### Ask Initial Questions over a Given Year (2020)

Let's first ask some questions over the UBER 10-k for 2020!

In [ ]:
response = index_set[2020].query("What were some of the biggest risk factors in 2020?", similarity_top_k=3)

In [ ]:
print(response)

In [ ]:
response = index_set[2020].query("What were some of the signifcant acquisitions?", similarity_top_k=3)

In [ ]:
print(response)

### Can a single vector index answer questions across years?

If we dump all documents to a single vector store, let's test its ability to answer questions across years!

In [ ]:
# # Option 2
# risk_query_str = (
#     "Describe the current risk factors. If the year is provided in the information, "
#     "provide that as well. If the context contains risk factors for multiple years, "
#     "explicitly provide the following:\n"
#     "- A description of the risk factors for each year\n"
#     "- A summary of how these risk factors are changing across years"
# )

In [ ]:
# Option 1
risk_query_str = "What are some of the biggest risk factors in each year?"

In [ ]:
response = global_index.query(risk_query_str, similarity_top_k=3)

In [ ]:
print(str(response))



The biggest risk factors in 2019 include economic downturns, natural disasters, geopolitical tensions, changes in government policies, risk-free interest rates, and the potential for rising inflation. Economic downturns can be caused by a variety of factors, such as a decrease in consumer spending, a decrease in business investment, or a decrease in exports. Natural disasters can cause significant damage to infrastructure and disrupt economic activity. Geopolitical tensions can lead to increased uncertainty and volatility in financial markets. Changes in government policies can have a significant impact on the economy, such as changes in taxation or regulations. Risk-free interest rates can affect the cost of borrowing and the availability of credit, while rising inflation can erode the purchasing power of consumers.


### Composing a Graph to synthesize answers across 10-K filings (2019-2022)

We want our queries to aggregate/synthesize information across *all* 10-K filings. To do this, we define a List index
on top of the 4 vector indices.

In [ ]:
from llama_index import GPTListIndex, LLMPredictor
from langchain import OpenAI
from llama_index.composability import ComposableGraph

In [ ]:
# set summary text for each doc
summaries = {}
for year in years:
    summaries[year] = f"UBER 10-k Filing for {year} fiscal year"

In [ ]:
# set number of output tokens
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, max_tokens=512))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

In [ ]:
graph = ComposableGraph.from_indices(
    GPTListIndex,
    [index_set[y] for y in years],
    [summaries[y] for y in years],
    service_context=service_context
)

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 0 tokens


In [ ]:
graph.save_to_disk('10k_graph.json')

In [ ]:
graph = ComposableGraph.load_from_disk('10k_graph.json', service_context=service_context)

### Setting Up the Query

We query about the risk factors. We want to synthesize information across each year.

In [ ]:
risk_query_str = (
    "Describe the current risk factors. If the year is provided in the information, "
    "provide that as well. If the context contains risk factors for multiple years, "
    "explicitly provide the following:\n"
    "- A description of the risk factors for each year\n"
    "- A summary of how these risk factors are changing across years"
)

In [ ]:
query_configs = [
    {
        "index_struct_type": "dict",
        "query_mode": "default",
        "query_kwargs": {
            "similarity_top_k": 1,
            # "include_summary": True
        }
    },
    {
        "index_struct_type": "list",
        "query_mode": "default",
        "query_kwargs": {
            "response_mode": "tree_summarize",
        }
    },
]

In [ ]:
response_summary = graph.query(risk_query_str, query_configs=query_configs)

In [ ]:
print(response_summary)


For 2017, 2018, and 2019, the risk factors included economic uncertainty, geopolitical tensions, and the potential for natural disasters.

For 2020, the risk factors include the COVID-19 pandemic and the impact of actions to mitigate the pandemic, which has adversely affected and continues to adversely affect business, financial condition, operating results, and prospects. This includes the potential for reduced demand for products and services, disruption of the supply chain, reduced liquidity, increased costs, and reduced revenue.

For 2021 and 2022, the risk factors include the potential for Drivers to be classified as employees, workers or quasi-employees instead of independent contractors, the highly competitive nature of the mobility, delivery, and logistics industries, and the need to lower fares or service fees and offer Driver incentives and consumer discounts and promotions in order to remain competitive in certain markets.

Overall, the risk factors have remained relatively

In [ ]:
print(response_summary.get_formatted_sources())

> Source (Doc id: 9b8c571e-b813-4906-9020-208cc57ac12a): UBER 10-k Filing for 2022 fiscal year...

> Source (Doc id: 8752ce30-d2c3-4ea6-9a5f-ca890105664b): year: 2022

announcements regarding our financial performance, including SEC filings, investor ev...

> Source (Doc id: 54649d63-ca22-4e40-a715-304606638bea): UBER 10-k Filing for 2021 fiscal year...

> Source (Doc id: 1a7abef5-2939-4d6d-aaa1-f579b57a65ea): year: 2021

our earnings calls and certain events we participate in or host with members of the i...

> Source (Doc id: a869db0a-254e-465d-836b-1e37335c2561): UBER 10-k Filing for 2020 fiscal year...

> Source (Doc id: 3cc53231-5406-4af3-8ab9-81fd61ffedc6): year: 2020

on Form 8-K and amendments to reports filed or furnished pursuant to Sections 13(a) a...

> Source (Doc id: b1538dd1-8270-45db-a603-ab6d34e9a6b3): UBER 10-k Filing for 2019 fiscal year...

> Source (Doc id: 4b64862e-b4bf-4013-86ab-ecaf6d4d0157): year: 2019

31,

2017

2018

2019

Risk-free interest...


In [ ]:
# query a specific year
response_tmp = index_set[2022].query(risk_query_str)

INFO:root:> [query] Total LLM token usage: 576 tokens
INFO:root:> [query] Total embedding token usage: 41 tokens


In [ ]:
str(response_tmp)

'\nIn 2022, the risk factors for our business include Drivers being classified as employees, workers or quasi-employees instead of independent contractors, the mobility, delivery, and logistics industries being highly competitive, and the need to lower fares or service fees and offer Driver incentives and consumer discounts and promotions in order to remain competitive in certain markets. Since our inception, we have incurred significant losses.'

In [ ]:
# query a global index
response = global_index.query(risk_query_str, similarity_top_k=4)

INFO:root:> [query] Total LLM token usage: 2926 tokens
INFO:root:> [query] Total embedding token usage: 41 tokens


In [ ]:
str(response)

"\n\nThe current risk factors include the COVID-19 pandemic and the impact of actions to mitigate the pandemic, the potential for Drivers to be classified as employees, workers or quasi-employees instead of independent contractors, and the highly competitive mobility, delivery, and logistics industries with well-established and low-cost alternatives, low barriers to entry, low switching costs, and well-capitalized competitors in nearly every major geographic region. To remain competitive, the company has in the past lowered, and may continue to lower, fares or service fees, and has in the past offered, and may continue to offer, significant Driver incentives and consumer discounts and promotions. Additionally, the risk-free interest rate has decreased from 2017 to 2019, from 2.4% to 2.1%, which could have an impact on the company's ability to access capital. Furthermore, the company has incurred significant losses since inception, including in the United States and other major markets,

In [2]:
%pip install -U gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.3 MB/s eta 0:

In [19]:
%pip  install -U torch transformers -q  accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
HuggingFaceLLM()

In [49]:
from pathlib import Path
import gradio as gr
import sys
import logging
import os
import torch

from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


from llama_index import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, load_index_from_storage, StorageContext

storage_path = "storage/"

docs_path="docs"

def construct_index(directory_path):
    max_input_size = 4096
    num_outputs = 512
    #max_chunk_overlap = 20
    chunk_overlap_ratio = 0.1
    chunk_size_limit = 600

    #prompt_helper = PromptHelper(max_input_size, num_outputs, chunk_overlap_ratio, chunk_size_limit=chunk_size_limit)

    system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
    - StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
    - StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
    - StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
    - StableLM will refuse to participate in anything that could harm a human.
    """

    # This will wrap the default prompts that are internal to llama-index
    query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")


    llm = HuggingFaceLLM()
    #     context_window=4096,
    #     max_new_tokens=256,
    #     generate_kwargs={"temperature": 0.7, "do_sample": False},
    #     system_prompt=system_prompt,
    #     query_wrapper_prompt=query_wrapper_prompt,
    #     tokenizer_name="StabilityAI/stablelm-tuned-alpha-3b",
    #     model_name="StabilityAI/stablelm-tuned-alpha-3b",
    #     device_map="auto",
    #     stopping_ids=[50278, 50279, 50277, 1, 0],
    #     tokenizer_kwargs={"max_length": 4096},
    #     # uncomment this if using CUDA to reduce memory usage
    #     # model_kwargs={"torch_dtype": torch.float16}
    #     # ,low_cpu_mem_usage=True
    # )
    #llm=ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo", max_tokens=num_outputs)
    #llm_predictor = LLMPredictor(llm=llm)
    service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm)

    documents = SimpleDirectoryReader(directory_path).load_data()

    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    #index = VectorStoreIndex(documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper)

    index.storage_context.persist(persist_dir=storage_path)

    return index

def chatbot(input_text):
    index = load_index_from_storage(StorageContext.from_defaults(persist_dir=storage_path))
    #index = GPTVectorStoreIndex.load_from_disk('index.json')
    #query_engine = index.as_query_engine(response_synthesizer=response_synthesizer);
    query_engine = index.as_query_engine(streaming=True)

    response = query_engine.query(input_text)

    print(response.source_nodes)

    relevant_files=[]

    for node_with_score in response.source_nodes:
        print(node_with_score)
        print(node_with_score.node)
        print(node_with_score.node.metadata)
        print(node_with_score.node.metadata['file_name'])

        file = node_with_score.node.metadata['file_name']
        print( file )

        # Resolve the full file path for the downloading
        full_file_path = Path( docs_path, file ).resolve()

        # See if it's already in the array
        if full_file_path not in relevant_files:
            relevant_files.append( full_file_path ) # Add it

    print( relevant_files )

    return response.get_response(), relevant_files

iface = gr.Interface(fn=chatbot,
                     inputs=gr.components.Textbox(lines=7, label="Enter your text"),
                     outputs=[
                        gr.components.Textbox(label="Response"),
                        gr.components.File(label="Relevant Files")
                        ],
                     title="Custom-trained AI Chatbot",
                     allow_flagging="never")

index = construct_index(docs_path)
iface.launch(share=False)

ImportError: ignored

In [45]:
%pip show accelerate

Name: accelerate
Version: 0.23.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, torch
Required-by: 


In [33]:
%pip install accelerate -U

In [ ]:
from pathlib import Path
import gradio as gr
import sys
import logging
import os
import torch
from accelerate import Accelerator

from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt
from transformers import AutoModelForCausalLM, AutoTokenizer

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


from llama_index import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, load_index_from_storage, StorageContext

storage_path = "storage"

docs_path="docs"

print(storage_path)

max_input_size = 4096
num_outputs = 512
#max_chunk_overlap = 20
chunk_overlap_ratio = 0.1
chunk_size_limit = 600


system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

model_name = "StabilityAI/stablelm-tuned-alpha-3b"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name)


model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-6b3", torch_dtype=torch.float16,
                                             use_cache=True, low_cpu_mem_usage=True)

import os
import tempfile

offload_dir='/content/offload'
os.makedirs(offload_dir) if not os.path.exists(offload_dir) else None

with tempfile.TemporaryDirectory() as tmp_dir:
    model.save_pretrained(tmp_dir, max_shard_size="200MB")
    print('Temp Dir Path:', tmp_dir)
    print(sorted(os.listdir(tmp_dir)))
    new_model = AutoModelForCausalLM.from_pretrained(tmp_dir, low_cpu_mem_usage=True, device_map="auto", offload_folder=offload_dir)



llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="StabilityAI/stablelm-tuned-alpha-3b",
    model_name="StabilityAI/stablelm-tuned-alpha-3b",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16},

)

service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm, embed_model="local")

documents = SimpleDirectoryReader(docs_path).load_data()

index = VectorStoreIndex.from_documents(documents, service_context=service_context)


def chatbot(input_text):
    query_engine = index.as_query_engine()

    response = query_engine.query(input_text)

    print(response.source_nodes)

    relevant_files=[]

    for node_with_score in response.source_nodes:
        print(node_with_score)
        print(node_with_score.node)
        print(node_with_score.node.metadata)
        print(node_with_score.node.metadata['file_name'])

        file = node_with_score.node.metadata['file_name']
        print( file )

        # Resolve the full file path for the downloading
        full_file_path = Path( docs_path, file ).resolve()

        # See if it's already in the array
        if full_file_path not in relevant_files:
            relevant_files.append( full_file_path ) # Add it

    print( relevant_files )

    return response.response, relevant_files

iface = gr.Interface(fn=chatbot,
                     inputs=gr.components.Textbox(lines=7, label="Enter your text"),
                     outputs=[
                        gr.components.Textbox(label="Response"),
                        gr.components.File(label="Relevant Files")
                        ],
                     title="Custom-trained AI Chatbot",
                     allow_flagging="never")

iface.launch(share=False)

storage


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]